In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder,StandardScaler

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
from sklearn.compose import make_column_transformer
import multiprocessing

from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [2]:
TEST_5=pd.read_excel("TEST-5.xlsx")
Well_D1=pd.read_excel("well_D1.xlsx")

In [3]:
Well_D1=Well_D1.drop(['LOI','V', 'Ni', 'Cu', 'Zn', 'Sr', 'Zr', 'Ba'], axis=1)

In [4]:
Well_D1.loc[:,'Mo'] = Well_D1.loc[:,'Mo'].mul(10000)

### Categorical features to numerical

In [5]:
def ordinal_encoder(test):
    well_encoder = OrdinalEncoder(categories=[test['WELL'].unique()])

    well_encoder.fit(test[["WELL"]])
    test["Well"] = well_encoder.transform(test[["WELL"]])
    test.drop('WELL', axis=1, inplace= True)
    
    Well_D1["Well"] = test['Well'].nunique() 

In [6]:
ordinal_encoder(TEST_5)

In [7]:
D1_depth=Well_D1.loc[:,'Depth']

In [8]:
X_values=TEST_5.drop(['MnO','SO3', 'Ga', 'Nb', 'Pb', 'Rb', 'Th', 'U', 'Y', 'Cr', 'Ba','Sr','Cu','Ni','V', 'Zn', 'Zr'], axis=1)
y_values=TEST_5.drop(['Depth','Al2O3', 'SiO2', 'TiO2', 'Fe2O3', 'MgO', 'CaO', 'Na2O', 'K2O','P2O5','Mo','Well'],axis=1)

## Random Forest

In [9]:
wells_scores = pd.DataFrame()
pred = pd.DataFrame(D1_depth)
        
for col in y_values:
    X = X_values
    y = y_values[col]
        
                
    X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )        
    
    # Evaluated Hyperparameters
    
    param_grid = {'n_estimators': [30,70,150],
                    'max_features': [5, 10, 25],
                    'max_depth'   : [None, 3, 10, 20]
                    }

    # Grid
    
    grid = GridSearchCV(
            estimator  = RandomForestRegressor(random_state = 123),
            param_grid = param_grid,
            scoring    = 'neg_root_mean_squared_error',
            n_jobs     = - 1,
            cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
            refit      = True,
            verbose    = 0,
            return_train_score = True
            )

    grid.fit(X = X_train, y = y_train)

    print("Final model best hyperparameters: ")        
    print("")       
    print(grid.best_params_, ":", grid.best_score_, grid.scoring)
    print("")
    
    # Test scores
        
    final_model = grid.best_estimator_
    predictions = final_model.predict(X = X_test)
        
    rmse = mean_squared_error(
            y_true  = y_test,
            y_pred  = predictions,
            squared = False
            )
                
    nrmse = rmse/(y_test.max()-y_test.min())
        
    R2 = r2_score(y_test, predictions)        
        
    print("Wells Test Scores")
    print(f"{col}:")
    print(f"RMSE: {rmse}")
    print(f"NRMSE: {nrmse}")        
    print(f"R2 score: {R2}")
    print("")
    print("")
        
                  
    # Save well test validation
                
    wells_scores = wells_scores.append({"y_hat": col, f"rmse_RF": rmse, f"nrmse_RF": nrmse, f"R2_RF": R2}, ignore_index=True)
                
    wells_scores.to_excel(f"Wells_TE_scores_RF_D1.xlsx")         
    
        
    # WELL D1 MAYOR ELEMENTS PREDICTIONS 
                
    X_test_D1 = Well_D1
    y_test_D1 = y_values[col]
    
    D1_predictions = final_model.predict(X_test_D1) 
    
                   
    # Save D1 predicted curves
        
    D1_pred = pd.DataFrame()

    D1_pred = D1_pred.assign(Predictions = D1_predictions.flatten().tolist())        
                
    pred=pd.concat([pred,D1_pred],axis=1)
        
    pred.columns=pred.columns.str.replace('Predictions', f"{col}")
        
    pred.to_excel(f"D1_TE_pred_RF.xlsx")       
              
    print("")
                
print(f"Well test validation")
print(wells_scores)

Final model best hyperparameters: 

{'max_depth': None, 'max_features': 10, 'n_estimators': 30} : -0.009763551535815325 neg_root_mean_squared_error

Wells Test Scores
MnO:
RMSE: 0.015635893011635593
NRMSE: 0.1145822439662582
R2 score: 0.34544439444019115



Final model best hyperparameters: 

{'max_depth': 10, 'max_features': 5, 'n_estimators': 70} : -0.721921726960892 neg_root_mean_squared_error

Wells Test Scores
SO3:
RMSE: 0.5624308104156124
NRMSE: 0.15902520694640046
R2 score: 0.27652257456649687



Final model best hyperparameters: 

{'max_depth': 10, 'max_features': 5, 'n_estimators': 70} : -812.047316401264 neg_root_mean_squared_error

Wells Test Scores
Ba:
RMSE: 2738.8582934458054
NRMSE: 0.11980798734460035
R2 score: 0.2067938422772081



Final model best hyperparameters: 

{'max_depth': None, 'max_features': 25, 'n_estimators': 150} : -23.31498449129773 neg_root_mean_squared_error

Wells Test Scores
Cr:
RMSE: 11.676243907494838
NRMSE: 0.0856538637073824
R2 score: 0.85381987710

# XGBoost

In [10]:
cat_col= ['Well']
categorical = ['c' if col in cat_col else 'q' for col in X_values.columns]

In [11]:
wells_scores = pd.DataFrame()
pred = pd.DataFrame(D1_depth)
        
for col in y_values:
    X = X_values
    y = y_values[col]
        
                
    X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )        
    
    # Evaluated Hyperparameters    
    
    param_grid = {'max_depth'        : [None, 1, 3, 5, 10, 20],
                  'subsample'        : [0.5, 1],
                  'learning_rate'    : [0.001, 0.01, 0.1],
                  'booster'          : ['gbtree']
                 }


    # Creation of validation set
    
    np.random.seed(123)
    idx_validacion = np.random.choice(
                        X_train.shape[0],
                        size=int(X_train.shape[0]*0.1), 
                        replace=False
                        )

    X_val = X_train.iloc[idx_validacion, :].copy()
    y_val = y_train.iloc[idx_validacion].copy()

    X_train_grid = X_train.reset_index(drop = True).drop(idx_validacion, axis = 0).copy()
    y_train_grid = y_train.reset_index(drop = True).drop(idx_validacion, axis = 0).copy()

        
    fit_params = {"eval_set": [(X_val, y_val)],
                   "verbose": False
                    }

    # Grid
    
    grid = GridSearchCV(
            estimator  = XGBRegressor(
                            n_estimators          = 1000,
                            early_stopping_rounds = 5,
                            eval_metric           = "rmse",
                            tree_method           ='hist',
                            enable_categorical    =True,
                            random_state          = 123,
                            feature_types         = categorical,
                        ),
            param_grid = param_grid,
            scoring    = 'neg_root_mean_squared_error',
            n_jobs     = multiprocessing.cpu_count() - 1,
            cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
            refit      = True,
            verbose    = 0,
            return_train_score = True
            )

    grid.fit(X = X_train_grid, y = y_train_grid, **fit_params)

    print("Final model best hyperparameters: ")        
    print("")       
    print(grid.best_params_, ":", grid.best_score_, grid.scoring)
    print("")
    n_trees_incluid = len(grid.best_estimator_.get_booster().get_dump())
    print(f"Final trees early stopping: {n_trees_incluid}")
    print("")
    
    # Test scores       
    
    final_model = grid.best_estimator_
    predictions = final_model.predict(X_test)
        
    rmse = mean_squared_error(
            y_true  = y_test,
            y_pred  = predictions,
            squared = False
               )
                
    nrmse = rmse/(y_test.max()-y_test.min())
        
    R2 = r2_score(y_test, predictions)        
        
    print("Wells Test Scores")
    print(f"{col}:")
    print(f"RMSE: {rmse}")
    print(f"NRMSE: {nrmse}")        
    print(f"R2 score: {R2}")
    print("")
    print("")
    
    # Save well test validation
                
    wells_scores = wells_scores.append({"y_hat":col,f"rmse_XGB": rmse, f"nrmse_XGB": nrmse, f"R2_XGB": R2}, ignore_index=True)
                
    wells_scores.to_excel(f"Wells_TE_scores_XGB_D1.xlsx") 
        
    
    # WELL D1 MAYOR ELEMENTS PREDICTIONS 
                
    X_test_D1 = Well_D1
    y_test_D1 = y_values[col]
    
    D1_predictions = final_model.predict(X_test_D1) 
    
       
    # Save D1 predicted curves
        
    D1_pred = pd.DataFrame()

    D1_pred = D1_pred.assign(Predictions = D1_predictions.flatten().tolist())        
                
    pred = pd.concat([pred,D1_pred],axis=1)
        
    pred.columns = pred.columns.str.replace('Predictions', f"{col}")
        
    pred.to_excel(f"D1_TE_pred_XGB.xlsx")  
    
    print("")
                
print(f"Well test validation")
print(wells_scores)    

Final model best hyperparameters: 

{'booster': 'gbtree', 'learning_rate': 0.1, 'max_depth': 3, 'subsample': 1} : -0.010037834485993274 neg_root_mean_squared_error

Final trees early stopping: 40

Wells Test Scores
MnO:
RMSE: 0.013998528627614255
NRMSE: 0.10258338434423461
R2 score: 0.4753545076859823



Final model best hyperparameters: 

{'booster': 'gbtree', 'learning_rate': 0.01, 'max_depth': 3, 'subsample': 1} : -0.7894856718622097 neg_root_mean_squared_error

Final trees early stopping: 418

Wells Test Scores
SO3:
RMSE: 0.22488469609905262
NRMSE: 0.06358530627047863
R2 score: 0.884333769981063



Final model best hyperparameters: 

{'booster': 'gbtree', 'learning_rate': 0.1, 'max_depth': 10, 'subsample': 0.5} : -813.6876459004519 neg_root_mean_squared_error

Final trees early stopping: 23

Wells Test Scores
Ba:
RMSE: 2987.285203235777
NRMSE: 0.1306751169567458
R2 score: 0.056373069374873075



Final model best hyperparameters: 

{'booster': 'gbtree', 'learning_rate': 0.01, 'max_d

## Gradient Boosting: HistGradientBoostingregressor

In [12]:
X_values['Well']=X_values['Well'].astype('category')

In [13]:
Well_D1_=Well_D1.copy()

cols = list(Well_D1_.columns)
cols = [cols[-1]] + cols[:-1]
Well_D1_ = Well_D1_[cols]

In [14]:
wells_scores = pd.DataFrame()
pred = pd.DataFrame(D1_depth)
        
for col in y_values:
    X = X_values
    y = y_values[col]
        
                
    X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )
    
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
    
    preprocessor = make_column_transformer(
                        (
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            cat_cols
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                   ).set_output(transform="pandas")
        
    X_train_prep = preprocessor.fit_transform(X_train)
    X_test_prep = preprocessor.transform(X_test)


    # Evaluated Hyperparameters
    
    param_grid = {'loss'             : ['squared_error', 'absolute_error'],
                    'learning_rate'    : [0.001, 0.01, 0.1],
                    'max_depth'        : [3, 5, 10, 20],
                    'l2_regularization': [0, 1, 10]
                    }

    # Grid
        
    grid = GridSearchCV(
            estimator  = HistGradientBoostingRegressor(
                            max_iter            = 1000, 
                            random_state        = 123,
                            early_stopping      = True,
                            validation_fraction = 0.1,
                            n_iter_no_change    = 10,
                            tol                 = 1e-7,
                            scoring             = 'loss',
                            categorical_features = cat_cols
                        ),
            param_grid = param_grid,
            scoring    = 'neg_root_mean_squared_error',
            n_jobs     = multiprocessing.cpu_count() - 1,
            cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
            refit      = True,
            verbose    = 0,
            return_train_score = True
            )

    grid.fit(X = X_train_prep, y = y_train)
        
        
    print("Final model best hyperparameters: ")        
    print("")       
    print(grid.best_params_, ":", grid.best_score_, grid.scoring)
    print("")        
    print(f"Final trees early stopping: {grid.best_estimator_.n_iter_}")
    print("")    
        
        
    # Test scores
    
    final_model = grid.best_estimator_
    predictions = final_model.predict(X = X_test_prep)
    
    rmse = mean_squared_error(
            y_true  = y_test,
            y_pred  = predictions,
            squared = False
           )
        
    nrmse = rmse/(y_test.max()-y_test.min())
        
    R2 = r2_score(y_test, predictions)        
        
    print("Wells Test Scores")
    print(f"{col}:")
    print(f"RMSE: {rmse}")
    print(f"NRMSE: {nrmse}")        
    print(f"R2 score: {R2}")
    print("")
    print("")
        
                  
    # Save well test validation
        
    wells_scores = wells_scores.append({"y_hat":col,f"rmse_HGBT": rmse, f"nrmse_HGBT": nrmse,f"R2_HGBT": R2},ignore_index=True)
                
    wells_scores.to_excel(f"Wells_TE_scores_HGBT_D1.xlsx")  
    
           
    # WELL D1 MAYOR ELEMENTS PREDICTIONS 
                
    X_test_D1 = Well_D1_
    y_test_D1 = y_values[col]
        
    D1_predictions = final_model.predict(X_test_D1) 
    
    # Save D1 predicted curves
        
    D1_pred = pd.DataFrame()

    D1_pred = D1_pred.assign(Predictions = D1_predictions.flatten().tolist())        
                
    pred = pd.concat([pred,D1_pred],axis=1)
        
    pred.columns = pred.columns.str.replace('Predictions', f"{col}")
        
    pred.to_excel(f"D1_TE_pred_HGBT.xlsx")        
    
    print("")
        
print(f"Well test validation")
print(wells_scores)
    

Final model best hyperparameters: 

{'l2_regularization': 0, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 3} : -0.01212398886682049 neg_root_mean_squared_error

Final trees early stopping: 121

Wells Test Scores
MnO:
RMSE: 0.015286247094842309
NRMSE: 0.11201998457307863
R2 score: 0.3743911018073669



Final model best hyperparameters: 

{'l2_regularization': 10, 'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 3} : -0.850907560407022 neg_root_mean_squared_error

Final trees early stopping: 403

Wells Test Scores
SO3:
RMSE: 0.4113019656629128
NRMSE: 0.11629409163888577
R2 score: 0.613091431959201



Final model best hyperparameters: 

{'l2_regularization': 0, 'learning_rate': 0.1, 'loss': 'absolute_error', 'max_depth': 3} : -938.4018712746971 neg_root_mean_squared_error

Final trees early stopping: 41

Wells Test Scores
Ba:
RMSE: 2918.063911970173
NRMSE: 0.12764711671015677
R2 score: 0.09959779303575844



Final model best hyperparameters: 

{'l2_regularizatio

# LightGBM

In [15]:
Well_D1['Well']=Well_D1['Well'].astype('category')

In [16]:
wells_scores = pd.DataFrame()
pred = pd.DataFrame(D1_depth)
    
for col in y_values:
    X = X_values
    y = y_values[col]
        
                
    X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )        
    
    # Evaluated Hyperparameters
    
    param_grid = {'n_estimators'     : [100, 500, 1000, 5000],
                  'max_depth'        : [-1, 1, 3, 5, 10, 20],
                  'subsample'        : [0.5, 1],
                  'learning_rate'    : [0.001, 0.01, 0.1],
                  'boosting_type'    : ['gbdt']
                     }

    # Grid
        
    grid = GridSearchCV(
            estimator  = LGBMRegressor(random_state=123),
            param_grid = param_grid,
            scoring    = 'neg_root_mean_squared_error',
            n_jobs     = multiprocessing.cpu_count() - 1,
            cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
            refit      = True,
            verbose    = 0,
            return_train_score = True
            )

    grid.fit(X = X_train, y = y_train, categorical_feature='auto')

    print("Final model best hyperparameters: ")        
    print("")       
    print(grid.best_params_, ":", grid.best_score_, grid.scoring)
    print("")
    
        
    # Test scores
        
    final_model = grid.best_estimator_
    predictions = final_model.predict(X = X_test)
        
    rmse = mean_squared_error(
            y_true  = y_test,
            y_pred  = predictions,
            squared = False
            )
        
    nrmse = rmse/(y_test.max()-y_test.min())
        
    R2 = r2_score(y_test, predictions)        
        
    print("Wells Test Scores")
    print(f"{col}:")
    print(f"RMSE: {rmse}")
    print(f"NRMSE: {nrmse}")        
    print(f"R2 score: {R2}")
    print("")
    print("")
        
                  
    # Save well test validation
                
    wells_scores = wells_scores.append({"y_hat":col,f"rmse_LGBM": rmse, f"nrmse_LGBM": nrmse, f"R2_LGBM": R2}, ignore_index=True)
                
    wells_scores.to_excel(f"Wells_TE_scores_LGBM_D1.xlsx")         
    
        
    # WELL D1 MAYOR ELEMENTS PREDICTIONS
    
    X_test_D1 = Well_D1
    y_test_D1 = y_values[col]
    
    D1_predictions = final_model.predict(X_test_D1) 
    
    # Save D1 predicted curves
        
    D1_pred = pd.DataFrame()

    D1_pred = D1_pred.assign(Predictions = D1_predictions.flatten().tolist())        
                
    pred = pd.concat([pred,D1_pred],axis=1)
        
    pred.columns = pred.columns.str.replace('Predictions', f"{col}")
        
    pred.to_excel(f"D1_TE_pred_LGBM.xlsx") 
        
        
    print("")
        
print(f"Well test validation")
print(wells_scores)
    

Final model best hyperparameters: 

{'boosting_type': 'gbdt', 'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 1000, 'subsample': 0.5} : -0.011965929500605703 neg_root_mean_squared_error

Wells Test Scores
MnO:
RMSE: 0.014392898584478576
NRMSE: 0.10547338842502255
R2 score: 0.445377228415334



Final model best hyperparameters: 

{'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 100, 'subsample': 0.5} : -0.7897639277009029 neg_root_mean_squared_error

Wells Test Scores
SO3:
RMSE: 0.3551458728524035
NRMSE: 0.10041616654105293
R2 score: 0.7115302213393639



Final model best hyperparameters: 

{'boosting_type': 'gbdt', 'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 500, 'subsample': 0.5} : -942.5881938866818 neg_root_mean_squared_error

Wells Test Scores
Ba:
RMSE: 2732.9669611632517
NRMSE: 0.1195502782600376
R2 score: 0.2102025729575886



Final model best hyperparameters: 

{'boosting_type': 'gbdt', 'learning_rate': 0.001, 'max_depth': -1, '